In [9]:
import pygame
import numpy as np
import random
from sys import exit
from Agent.TriviaLC import triviaAgent
from Agent.HangmanLC import HangMan

Ashkan

In [ ]:
# Initialize agents
trivia_agent = triviaAgent()
hangman_agent = HangMan()

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((800, 800))
pygame.display.set_caption("Maropelleh (Snakes, Ladders, Trivia, Hangman)")
clock = pygame.time.Clock()

# Load assets
sky_background = pygame.image.load('graphics/blue-sky.png')

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
GRAY = (200, 200, 200)
YELLOW = (255, 255, 0)
LIGHT_YELLOW = (255, 255, 200)
ORANGE = (255, 200, 0)

# Game constants
BOARD_SIZE = 10
CELL_SIZE = 60
GRID_OFFSET = 100
PLAYER_SIZE = 15
BUTTON_WIDTH = 150
BUTTON_HEIGHT = 50

# Game state
player1_pos = 0
player2_pos = 0
current_player = 1
game_over = False
winner = None
message = "Player 1's turn. Press the button to roll!"
message_timer = 0
message_duration = 120  # in frames

# Mini-game modal
active_modal = None  # 'trivia', 'hangman', or None
modal_data = None

# Random snakes and ladders
snakes = {}
ladders = {}
for _ in range(5):
    start = random.randint(20, 98)
    end = random.randint(1, start - 1)
    snakes[end] = start

for _ in range(5):
    start = random.randint(1, 80)
    end = random.randint(start + 1, 99)
    ladders[start] = end

# Assign tiles to mini-games
total_cells = list(range(1, 100))
random.shuffle(total_cells)
trivia_cells = set(total_cells[:15])
hangman_cells = set(total_cells[15:30])

# Fonts
font = pygame.font.SysFont(None, 36)
small_font = pygame.font.SysFont(None, 24)

def draw_board():
    screen.fill(WHITE)
    screen.blit(sky_background, (0, 0))

    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            cell_num = row * BOARD_SIZE + col + 1
            x = col * CELL_SIZE + GRID_OFFSET
            y = (BOARD_SIZE - 1 - row) * CELL_SIZE + GRID_OFFSET

            if cell_num in trivia_cells:
                color = BLUE
            elif cell_num in hangman_cells:
                color = YELLOW
            else:
                color = LIGHT_YELLOW if (row + col) % 2 == 0 else ORANGE

            pygame.draw.rect(screen, color, (x, y, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, BLACK, (x, y, CELL_SIZE, CELL_SIZE), 1)

            text = font.render(str(cell_num), True, BLACK)
            text_rect = text.get_rect(center=(x + CELL_SIZE//2, y + CELL_SIZE//2))
            screen.blit(text, text_rect)

            if cell_num in snakes:
                pygame.draw.circle(screen, RED, (x + CELL_SIZE//2, y + CELL_SIZE//2), 5)
                label = small_font.render("S", True, RED)
                screen.blit(label, label.get_rect(center=(x + CELL_SIZE//2, y + CELL_SIZE//2 + 15)))
            elif cell_num in ladders:
                pygame.draw.circle(screen, GREEN, (x + CELL_SIZE//2, y + CELL_SIZE//2), 5)
                label = small_font.render("L", True, GREEN)
                screen.blit(label, label.get_rect(center=(x + CELL_SIZE//2, y + CELL_SIZE//2 + 15)))
            elif cell_num in trivia_cells:
                label = small_font.render("T", True, WHITE)
                screen.blit(label, label.get_rect(center=(x + CELL_SIZE//2, y + CELL_SIZE//2 + 15)))
            elif cell_num in hangman_cells:
                label = small_font.render("H", True, BLACK)
                screen.blit(label, label.get_rect(center=(x + CELL_SIZE//2, y + CELL_SIZE//2 + 15)))

def get_cell_position(pos):
    row = (pos - 1) // BOARD_SIZE
    col = (pos - 1) % BOARD_SIZE
    x = col * CELL_SIZE + GRID_OFFSET + CELL_SIZE//2
    y = (BOARD_SIZE - 1 - row) * CELL_SIZE + GRID_OFFSET + CELL_SIZE//2
    return x, y

def draw_players():
    x1, y1 = get_cell_position(player1_pos + 1)
    pygame.draw.circle(screen, RED, (x1 - 10, y1), PLAYER_SIZE)
    x2, y2 = get_cell_position(player2_pos + 1)
    pygame.draw.circle(screen, BLUE, (x2 + 10, y2), PLAYER_SIZE)

def draw_button():
    button_rect = pygame.Rect(330, 720, BUTTON_WIDTH, BUTTON_HEIGHT)
    pygame.draw.rect(screen, YELLOW, button_rect)
    pygame.draw.rect(screen, BLACK, button_rect, 2)
    text = font.render("Roll Dice", True, BLACK)
    screen.blit(text, text.get_rect(center=button_rect.center))
    return button_rect

def draw_game_info():
    screen.blit(font.render(f"Current Player: {current_player}", True, BLACK), (20, 20))
    if game_over:
        screen.blit(font.render(f"Player {winner} wins!", True, BLACK), (20, 50))
    elif message_timer > 0:
        screen.blit(font.render(message, True, BLACK), (20, 70))

def draw_modal():
    if active_modal and modal_data:
        modal_rect = pygame.Rect(150, 250, 500, 300)
        pygame.draw.rect(screen, GRAY, modal_rect)
        pygame.draw.rect(screen, BLACK, modal_rect, 3)

        lines = modal_data.split("\n")
        for i, line in enumerate(lines):
            line_surf = small_font.render(line.strip(), True, BLACK)
            screen.blit(line_surf, (modal_rect.x + 20, modal_rect.y + 40 + 25 * i))

        exit_msg = small_font.render("Press SPACE to continue...", True, BLACK)
        screen.blit(exit_msg, (modal_rect.x + 20, modal_rect.y + 250))

def roll_dice():
    return random.randint(1, 6)

def move_player(player_pos, steps):
    new_pos = player_pos + steps
    if new_pos in ladders:
        old_pos = new_pos
        new_pos = ladders[new_pos]
        return new_pos, f"Player {current_player} climbed a ladder from {old_pos+1} to {new_pos+1}!"
    if new_pos in snakes:
        old_pos = new_pos
        new_pos = snakes[new_pos]
        return new_pos, f"Player {current_player} was bitten by a snake from {old_pos+1} to {new_pos+1}!"
    if new_pos > 99:
        new_pos = 99
    return new_pos, f"Player {current_player} moved {steps} steps to {new_pos+1}"

def trigger_mini_game(cell):
    global active_modal, modal_data
    if cell in trivia_cells:
        active_modal = "trivia"
        response = trivia_agent.envoke("general knowledge")
        modal_data = response if isinstance(response, str) else str(response)
    elif cell in hangman_cells:
        active_modal = "hangman"
        response = hangman_agent.envoke()
        modal_data = f"Guess this word: {response if isinstance(response, str) else str(response)}"

# Main Game Loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()

        if active_modal and event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            active_modal = None
            modal_data = None

        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over and not active_modal:
            mouse_pos = pygame.mouse.get_pos()
            if draw_button().collidepoint(mouse_pos):
                dice_roll = roll_dice()
                if current_player == 1:
                    player1_pos, move_message = move_player(player1_pos, dice_roll)
                    if player1_pos == 99:
                        game_over = True
                        winner = 1
                        message = f"Player 1 wins the game!"
                    trigger_mini_game(player1_pos + 1)
                else:
                    player2_pos, move_message = move_player(player2_pos, dice_roll)
                    if player2_pos == 99:
                        game_over = True
                        winner = 2
                        message = f"Player 2 wins the game!"
                    trigger_mini_game(player2_pos + 1)

                message = move_message
                message_timer = message_duration
                current_player = 3 - current_player

    if message_timer > 0:
        message_timer -= 1

    draw_board()
    draw_players()
    draw_game_info()
    draw_button()
    draw_modal()
    pygame.display.update()
    clock.tick(60)


2025-04-15 11:24:48.327 Python[90621:18234858] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/pd/wynthhk510v_lw3l_4q234gh0000gn/T/org.python.python.savedState
2025-04-15 11:24:48.836 Python[90621:18234858] +[IMKClient subclass]: chose IMKClient_Legacy
2025-04-15 11:24:48.836 Python[90621:18234858] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


SystemExit: 

/Users/rav007/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

Brandon

2025-04-14 23:37:56.335 Python[88742:18227719] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/pd/wynthhk510v_lw3l_4q234gh0000gn/T/org.python.python.savedState
2025-04-14 23:37:57.008 Python[88742:18227719] +[IMKClient subclass]: chose IMKClient_Legacy
2025-04-14 23:37:57.008 Python[88742:18227719] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


SystemExit: 

/Users/rav007/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 